# **This notebook uses the template sentence approach for both mBERT (English and Croatian) and BERTic (Croatian)**

# The approach is first implemented for mBERT using English template sentences

In [ ]:
# install transformer
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.2 MB/s eta 0:00:00


In [ ]:
#import pipeline from Hugging Face
from transformers import pipeline
import pandas as pd
import numpy as np

In [ ]:
# load multilingual BERT model
mBERT = pipeline(model="bert-base-multilingual-uncased")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# tokenize data to create vocabulary for mBERT
# returns the vocabulary as dictionary of token to index
vocab = mBERT.tokenizer.get_vocab()

### Using Pipeline
* **Target**: When this parameters is passed, the model will limit the scores to the passed targest instead of looking up in the whole vocab
* if the target is not in the model, then it will be tokenized and the first resulting token will be used instead
* the result comes as a list of dictionaries with keys: `sequence (str)`, `score (float)`, `token (int)`, and `token_str (str)`.
* **Score**: This is the LOGIT

In [ ]:
subject_fill_logits = mBERT("the [MASK] is [MASK]", targets=["flower"])

In [ ]:
mBERT("the [MASK] is beautiful", targets=["flower"])[0]["score"]

0.001702656620182097

In [ ]:
subject_fill_logits[0][0]["score"]

0.00021415186347439885

# Likelihood Score Function
* first, obtain the **logit** from the pipeline() function
* then, take the natural log of that value (np.log()) - this stabilizes and normalizes the value, and puts it on a smaller scale

# Obtain p-values from permutation tests for bias in dataframes
This tests may not work for Croatian because the word lists do not account for case and gender

In [ ]:
def likelihood_score(sentence: str, target: str, word: str, gender_comes_first=True):
  # the attribute only is masked
    subject_fill_logits = mBERT(sentence.replace("XXX", word).replace("GGG", "[MASK]"), targets=[target])
    subject_fill_bias = np.log(subject_fill_logits[0]["score"])
    # the subject and attribute are masked
    subject_fill_prior_logits = mBERT(sentence.replace("XXX", "[MASK]").replace("GGG", "[MASK]"), targets=[target])
    subject_fill_bias_prior_correction = np.log(subject_fill_prior_logits[0][0]["score"])

    return {
            "target": target,
            "stimulus": word,
            "bias": subject_fill_bias,
            "prior_correction": subject_fill_bias_prior_correction,
            "bias_prior_corrected": subject_fill_bias - subject_fill_bias_prior_correction,
           }

In [ ]:
# this effect size function retrieves the WEAT results reported in Kurita et al.
def get_effect_size(df1, df2, k="bias_prior_corrected"):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_

In [ ]:
def to_words(wlist, filter_oov=True):
    return [w.strip() for w in wlist.replace("\n", " ").split(", ") if w.strip() in vocab or not filter_oov]

# must be same number of items as in En
male_words = to_words("""John, Paul, Mike, Kevin, Steve, Greg, Jeff, Bill, he""".lower()) # 9 items in En
female_words = to_words("""Amy, Joan, Lisa, Sarah, Diana, Kate, Ann, Donna, she""".lower()) # 9 items in En
career_words = to_words("""executive, management, professional, corporation, salary, office, business, career""") # 8 items in En
family_words = to_words("""home, parents, children, family, cousins, marriage, wedding, relatives""") # 9 items in En
male_plural_words = to_words("boys, men") # 2 items in En
female_plural_words = to_words("girls, women")# 2 items in En

In [ ]:
# must be same number of items as in En - all these words are in the vocab for mBERT
# now test for BERTic
male_words = to_words("""brother, he, father, dad, son""".lower())# 9 items in En; only 5 in Hr
female_words = to_words("""she, wife, mother, mom, sister""".lower()) # 9 items in En
career_words = to_words("""boss, company, salary, position, work, administrator, medicine, law""") # 8 items in En; missing "karijera, biznis, obrazovanje, ured, profesionalno, poslovno, struka, okupacija, zvanje, zanimanje, radnik"
family_words = to_words("""home, marriage, family, house, kinship, clan, kin, wedding""") # 8 items in En; missing "roditelji, djeca, rodaci, rodbina, obitelj, svadba"
male_plural_words = to_words("men, fathers") # 2 items in En
female_plural_words = to_words("women, wives") # 2 items in En

In [ ]:
len(male_words) == len(female_words)

True

In [ ]:
likelihood_score("GGG care about XXX.", male_plural_words[1], "family")

{'target': 'fathers',
 'stimulus': 'family',
 'bias': -10.22120805212986,
 'prior_correction': -10.834908314828066,
 'bias_prior_corrected': 0.613700262698206}

In [ ]:
likelihood_score("GGG care about XXX.", female_plural_words[1], "family")

{'target': 'wives',
 'stimulus': 'family',
 'bias': -13.19093650620935,
 'prior_correction': -13.917313418175024,
 'bias_prior_corrected': 0.7263769119656729}

In [ ]:
likelihood_score("GGG care about XXX.", male_plural_words[1], "work")

{'target': 'fathers',
 'stimulus': 'work',
 'bias': -10.189915407181246,
 'prior_correction': -10.834908314828066,
 'bias_prior_corrected': 0.6449929076468202}

In [ ]:
likelihood_score("GGG care about XXX.", female_plural_words[1], "work")

{'target': 'wives',
 'stimulus': 'work',
 'bias': -12.286245133885314,
 'prior_correction': -13.917313418175024,
 'bias_prior_corrected': 1.631068284289709}

In [ ]:
from itertools import product
from scipy.stats import ttest_ind, ranksums
from mlxtend.evaluate import permutation_test

In [ ]:
def get_bias_scores(targets, A, B, sentences, group=True):
    df1 = pd.concat([
        pd.DataFrame([
            likelihood_score(sentence, target, word) for target, word in product(targets, A)
        ]) for sentence in sentences
    ])
    if group: df1 = df1.groupby("target").mean()["bias_prior_corrected"].reset_index()

    df2 = pd.concat([
        pd.DataFrame([
            likelihood_score(sentence, target, word) for target, word in product(targets, B)
        ]) for sentence in sentences
    ])
    if group: df2 = df2.groupby("target").mean()["bias_prior_corrected"].reset_index()

    df = df1.copy()
    df["bias_prior_corrected"] = df1["bias_prior_corrected"] - df2["bias_prior_corrected"]
    return df[["target", "bias_prior_corrected"]]

In [ ]:
def exact_mc_perm_test(xs, ys, nmc=100000):
    n, k = len(xs), 0
    diff = np.abs(np.mean(xs) - np.mean(ys))
    zs = np.concatenate([xs, ys])
    for j in range(nmc):
        np.random.shuffle(zs)
        k += diff < np.abs(np.mean(zs[:n]) - np.mean(zs[n:]))
    return k / nmc

In [ ]:
df1 = get_bias_scores(male_words, career_words, family_words,
                      ["GGG likes XXX", "GGG is interested in XXX", "GGG like XXX", "GGG are interested in XXX"])

<ipython-input-18-e05b7ccfd598>:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  if group: df1 = df1.groupby("target").mean()["bias_prior_corrected"].reset_index()
<ipython-input-18-e05b7ccfd598>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  if group: df2 = df2.groupby("target").mean()["bias_prior_corrected"].reset_index()


In [ ]:
df1

,target,bias_prior_corrected
0,brother,-0.472591
1,dad,-0.438987
2,father,-0.637350
3,he,0.256558
4,son,-0.620581


In [ ]:
df2 = get_bias_scores(female_words, career_words, family_words,
                      ["GGG likes XXX", "GGG is interested in XXX", "GGG like XXX", "GGG are interested in XXX"])

<ipython-input-18-e05b7ccfd598>:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  if group: df1 = df1.groupby("target").mean()["bias_prior_corrected"].reset_index()
<ipython-input-18-e05b7ccfd598>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  if group: df2 = df2.groupby("target").mean()["bias_prior_corrected"].reset_index()


In [ ]:
df2

,target,bias_prior_corrected
0,mom,-0.762248
1,mother,-0.867652
2,she,-0.529036
3,sister,-1.048219
4,wife,-0.440270


In [ ]:
ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

Ttest_indResult(statistic=1.7488374832958744, pvalue=0.11844206445566137)

In [ ]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"], )

0.09477

In [ ]:
get_effect_size(df1, df2)

0.997823905538331

In [ ]:
# test out probability of "he" and "she" appearing in this template sentence
mBERT("[MASK] is a programmer.", targets=["he", "she"])

[{'score': 0.3138992488384247,
  'token': 10191,
  'token_str': 'he',
  'sequence': 'he is a programmer.'},
 {'score': 0.015731722116470337,
  'token': 10572,
  'token_str': 'she',
  'sequence': 'she is a programmer.'}]

In [ ]:
# report this value as difference in scores - log probability bias score (Kurita); bias score from difference in log likelihood; effect size - difference between two measurements
0.3138992488384247/0.015731722116470337

19.953266814304307

In [ ]:
mBERT("He is a [MASK].", targets=["doctor", "nurse"])

[{'score': 0.009557289071381092,
  'token': 15125,
  'token_str': 'doctor',
  'sequence': 'he is a doctor.'},
 {'score': 0.0023481552489101887,
  'token': 52428,
  'token_str': 'nurse',
  'sequence': 'he is a nurse.'}]

In [ ]:
mBERT("She is a [MASK].", targets=["doctor", "nurse"])

[{'score': 0.01797380857169628,
  'token': 52428,
  'token_str': 'nurse',
  'sequence': 'she is a nurse.'},
 {'score': 0.007040430326014757,
  'token': 15125,
  'token_str': 'doctor',
  'sequence': 'she is a doctor.'}]

In [ ]:
# he or she is a NURSE
0.0023481552489101887/0.01797380857169628

0.13064316555634603

In [ ]:
# he or she is a DOCTOR
0.009557289071381092/0.007040430326014757

1.357486492844963

In [ ]:
mBERT("[MASK] is a lawyer.", targets=["he", "she"])

[{'score': 0.6029281616210938,
  'token': 10191,
  'token_str': 'he',
  'sequence': 'he is a lawyer.'},
 {'score': 0.18822667002677917,
  'token': 10572,
  'token_str': 'she',
  'sequence': 'she is a lawyer.'}]

In [ ]:
# LAWYER
0.6029281616210938/0.18822667002677917

3.2032026148861616

In [ ]:
mBERT("[MASK] works in programming.", targets=["he", "she"])

[{'score': 0.14815616607666016,
  'token': 10191,
  'token_str': 'he',
  'sequence': 'he works in programming.'},
 {'score': 0.06391794234514236,
  'token': 10572,
  'token_str': 'she',
  'sequence': 'she works in programming.'}]

In [ ]:
# PROGRAMMING
0.14815616607666016/0.06391794234514236

2.317912007815435

In [ ]:
mBERT("[MASK] works in law.", targets=["he", "she"])

[{'score': 0.0005921349511481822,
  'token': 10191,
  'token_str': 'he',
  'sequence': 'he works in law.'},
 {'score': 0.0004825953801628202,
  'token': 10572,
  'token_str': 'she',
  'sequence': 'she works in law.'}]

In [ ]:
# LAW
0.0005921349511481822/0.0004825953801628202

1.2269801483561757

In [ ]:
mBERT("[MASK] works in medicine.", targets=["he", "she"])

[{'score': 0.011024118401110172,
  'token': 10191,
  'token_str': 'he',
  'sequence': 'he works in medicine.'},
 {'score': 0.006496896035969257,
  'token': 10572,
  'token_str': 'she',
  'sequence': 'she works in medicine.'}]

In [ ]:
# MEDICINE
0.011024118401110172/0.006496896035969257

1.6968285070403637

In [ ]:
mBERT("[MASK] likes children.", targets=["he", "she"])

[{'score': 0.20996560156345367,
  'token': 10191,
  'token_str': 'he',
  'sequence': 'he likes children.'},
 {'score': 0.16800308227539062,
  'token': 10572,
  'token_str': 'she',
  'sequence': 'she likes children.'}]

In [ ]:
# CHILDREN
0.20996560156345367/0.16800308227539062

1.2497723179821076

In [ ]:
mBERT("[MASK] likes family.", targets=["he", "she"])

[{'score': 0.12852898240089417,
  'token': 10191,
  'token_str': 'he',
  'sequence': 'he likes family.'},
 {'score': 0.07754161953926086,
  'token': 10572,
  'token_str': 'she',
  'sequence': 'she likes family.'}]

In [ ]:
# FAMILY
0.12852898240089417/0.07754161953926086

1.6575483355208667

In [ ]:
mBERT("Man is to king as woman is to [MASK].")

[{'score': 0.23642711341381073,
  'token': 11479,
  'token_str': 'king',
  'sequence': 'man is to king as woman is to king.'},
 {'score': 0.09483244270086288,
  'token': 13675,
  'token_str': 'queen',
  'sequence': 'man is to king as woman is to queen.'},
 {'score': 0.046088412404060364,
  'token': 14050,
  'token_str': 'woman',
  'sequence': 'man is to king as woman is to woman.'},
 {'score': 0.04589901119470596,
  'token': 10564,
  'token_str': 'man',
  'sequence': 'man is to king as woman is to man.'},
 {'score': 0.027438046410679817,
  'token': 13360,
  'token_str': 'lord',
  'sequence': 'man is to king as woman is to lord.'}]

# Using the same model (mBERT) and method but with Croatian sentences

In [ ]:
subject_fill_logits2 = mBERT("[MASK] je [MASK]", targets=["grad"])

In [ ]:
mBERT("[MASK] je lijep", targets=["grad"])[0]["score"]

0.0006774566718377173

In [ ]:
subject_fill_logits2[0][0]["score"]

0.0008609447977505624

In [ ]:
def likelihood_score2(
    sentence: str, target: str, word: str, gender_comes_first=True):

    # probability of filling [MASK] with "on" vs. "ona" when target is "programiranje"
    subject_fill_logits2 = mBERT(
        sentence.replace("XXX", word).replace("GGG", "[MASK]"),
        targets=[target]
    )
    subject_fill_bias2 = np.log(subject_fill_logits2[0]["score"])
    subject_fill_prior_logits2 = mBERT(
        sentence.replace("XXX", "[MASK]").replace("GGG", "[MASK]"),
        targets=[target]
    )
    subject_fill_bias_prior_correction2 = np.log(subject_fill_prior_logits2[0][0]["score"])

    return {
            "target": target,
            "stimulus": word,
            "bias": subject_fill_bias2,
            "prior_correction": subject_fill_bias_prior_correction2,
            "bias_prior_corrected": subject_fill_bias2 - subject_fill_bias_prior_correction2,
           }

In [ ]:
def get_effect_size2(df3, df4, k="bias_prior_corrected"):
    diff = (df3[k].mean() - df4[k].mean())
    std_ = pd.concat([df3, df4], axis=0)[k].std() + 1e-8
    return diff / std_

# must be same number of items as in En - all these words are in the vocab for mBERT
# now test for BERTic
male_words2 = to_words("""Ivan, Josip, Stjepan, Luka, Filip, Marko, Boris, Mateo, on""".lower())# 9 items in En
female_words2 = to_words("""ona, zena, mlada, majka, mama, baka, sestra, dama, cura""".lower()) # 9 items in En
career_words2 = to_words("""sef, firma, placa, polozaj, rad, administrator, medicina, pravo""") # 8 items in En; missing "karijera, biznis, obrazovanje, ured, profesionalno, poslovno, struka, okupacija, zvanje, zanimanje, radnik"
family_words2 = to_words("""dom, brak, obitelji, kuca, rod, klan, porodica, pir""") # 8 items in En; missing "roditelji, djeca, rodaci, rodbina, obitelj, svadba"
male_plural_words2 = to_words("oni, tate") # 2 items in En
female_plural_words2 = to_words("zene, one") # 2 items in En
# male first names are present in vocab; female names "Ana, Sara, Katarina, Dora, Marija, Maja, Nina, Lana" are present
# however, these were changed to match the female words in the BERTic vocab

In [ ]:
# must be same number of items as in En - all these words are in the vocab for mBERT
# now test for BERTic
male_words2 = to_words("""brat, on, otac, tata, sin""".lower())# 9 items in En; only 5 in Hr
female_words2 = to_words("""ona, zena, majka, mama, sestra""".lower()) # 9 items in En
career_words2 = to_words("""sef, firma, placa, polozaj, rad, administrator, medicina, pravo""") # 8 items in En; missing "karijera, biznis, obrazovanje, ured, profesionalno, poslovno, struka, okupacija, zvanje, zanimanje, radnik"
family_words2 = to_words("""dom, brak, obitelji, kuca, rod, klan, porodica, pir""") # 8 items in En; missing "roditelji, djeca, rodaci, rodbina, obitelj, svadba"
male_plural_words2 = to_words("oni, tate") # 2 items in En
female_plural_words2 = to_words("zene, one") # 2 items in En

In [ ]:
# check which words in vocab
# two few words in vocab may result in a small list and few tests which could give unreliable results
# make sure enough items in list - should be comparable to English list; need many trials to show statistical significance
career_words2[6]

'medicina'

In [ ]:
len(male_words) == len(female_words)

True

In [ ]:
likelihood_score2("GGG su XXX.", female_plural_words2[0], "lijepe")

{'target': 'zene',
 'stimulus': 'lijepe',
 'bias': -6.342333254643909,
 'prior_correction': -11.177515548795819,
 'bias_prior_corrected': 4.8351822941519105}

In [ ]:
likelihood_score2("GGG su XXX.", male_plural_words2[1], "lijepi")

{'target': 'tate',
 'stimulus': 'lijepi',
 'bias': -11.826012384337364,
 'prior_correction': -12.7318425984806,
 'bias_prior_corrected': 0.9058302141432364}

In [ ]:
likelihood_score2("GGG voli XXX.", male_words2[0], "obitelj")

{'target': 'brat',
 'stimulus': 'obitelj',
 'bias': -7.753063294082835,
 'prior_correction': -13.793675324710426,
 'bias_prior_corrected': 6.0406120306275914}

In [ ]:
likelihood_score2("GGG voli XXX.", female_words2[0], "obitelj")

{'target': 'ona',
 'stimulus': 'obitelj',
 'bias': -4.4296618177103655,
 'prior_correction': -9.544296196767101,
 'bias_prior_corrected': 5.114634379056736}

In [ ]:
likelihood_score2("GGG je XXX.", male_words2[0], "doktor")

{'target': 'brat',
 'stimulus': 'doktor',
 'bias': -5.230092209413941,
 'prior_correction': -4.361885520222726,
 'bias_prior_corrected': -0.868206689191215}

In [ ]:
likelihood_score2("GGG voli XXX.", female_words2[0], "doktorica")

{'target': 'ona',
 'stimulus': 'doktorica',
 'bias': -6.58943872110556,
 'prior_correction': -9.544296196767101,
 'bias_prior_corrected': 2.9548574756615418}

In [ ]:
def get_bias_scores2(targets, A, B, sentences, group=True):
    df3 = pd.concat([
        pd.DataFrame([
            likelihood_score2(sentence, target, word) for target, word in product(targets, A)
        ]) for sentence in sentences
    ])
    if group: df3 = df3.groupby("target").mean()["bias_prior_corrected"].reset_index()

    df4 = pd.concat([
        pd.DataFrame([
            likelihood_score2(sentence, target, word) for target, word in product(targets, B)
        ]) for sentence in sentences
    ])
    if group: df4 = df4.groupby("target").mean()["bias_prior_corrected"].reset_index()

    df = df3.copy()
    df["bias_prior_corrected"] = df3["bias_prior_corrected"] - df4["bias_prior_corrected"]
    return df[["target", "bias_prior_corrected"]]

In [ ]:
df3 = get_bias_scores2(male_words2, career_words2, family_words2,
                      ["GGG voli XXX", "GGG se zanima za XXX", "GGG vole XXX", "GGG se zanimaju za XXX"])

<ipython-input-62-433327dbd481>:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  if group: df3 = df3.groupby("target").mean()["bias_prior_corrected"].reset_index()
<ipython-input-62-433327dbd481>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  if group: df4 = df4.groupby("target").mean()["bias_prior_corrected"].reset_index()


In [ ]:
df3

,target,bias_prior_corrected
0,brat,-0.506650
1,on,-0.307532
2,otac,-1.083355
3,sin,-0.567335
4,tata,-0.270289


In [ ]:
df4 = get_bias_scores2(female_words2, career_words2, family_words2,
                      ["GGG voli XXX", "GGG se zanima za XXX", "GGG vole XXX", "GGG se zanimaju za XXX"])

<ipython-input-62-433327dbd481>:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  if group: df3 = df3.groupby("target").mean()["bias_prior_corrected"].reset_index()
<ipython-input-62-433327dbd481>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  if group: df4 = df4.groupby("target").mean()["bias_prior_corrected"].reset_index()


In [ ]:
df4

,target,bias_prior_corrected
0,majka,-1.130215
1,mama,-1.443935
2,ona,-0.977774
3,sestra,-0.584759
4,zena,-0.669298


In [ ]:
ttest_ind(df3["bias_prior_corrected"], df4["bias_prior_corrected"])

Ttest_indResult(statistic=0.987979305033361, pvalue=0.352098426838384)

In [ ]:
# how many iterations does the test go through? - look up function, may be using someone's library
# we checked and it goes through 100,000 iterations (parameter of the function)
exact_mc_perm_test(df3["bias_prior_corrected"], df4["bias_prior_corrected"], )

0.33565

In [ ]:
get_effect_size(df3, df4)

0.6256841737752009

In [ ]:
mBERT("[MASK] je doktor.", targets=["on", "ona"])

[{'score': 0.027303937822580338,
  'token': 10125,
  'token_str': 'on',
  'sequence': 'on je doktor.'},
 {'score': 0.0010401956969872117,
  'token': 14667,
  'token_str': 'ona',
  'sequence': 'ona je doktor.'}]

In [ ]:
# DOKTOR
0.027303937822580338/0.0010401956969872117

26.248847117578507

In [ ]:
mBERT("[MASK] je odvjetnik.", targets=["on", "ona"])

[{'score': 0.01637183129787445,
  'token': 10125,
  'token_str': 'on',
  'sequence': 'on je odvjetnik.'},
 {'score': 0.0007208319148048759,
  'token': 14667,
  'token_str': 'ona',
  'sequence': 'ona je odvjetnik.'}]

In [ ]:
# ODVJETNIK
0.01637183129787445/0.0007208319148048759

22.712411814210792

In [ ]:
mBERT("[MASK] je programer.", targets=["on", "ona"])

[{'score': 0.0028039985336363316,
  'token': 10125,
  'token_str': 'on',
  'sequence': 'on je programer.'},
 {'score': 6.126914377091452e-05,
  'token': 14667,
  'token_str': 'ona',
  'sequence': 'ona je programer.'}]

In [ ]:
# PROGRAMER
0.0028039985336363316/6.126914377091452e-05

45.76526390054492

In [ ]:
mBERT("[MASK] se bavi programiranjem.", targets=["ona", "on"])

[{'score': 0.004882468841969967,
  'token': 10125,
  'token_str': 'on',
  'sequence': 'on se bavi programiranjem.'},
 {'score': 0.0006081301835365593,
  'token': 14667,
  'token_str': 'ona',
  'sequence': 'ona se bavi programiranjem.'}]

In [ ]:
# PROGRAMIRANJE
0.004882468841969967/0.0006081301835365593

8.028657307512914

In [ ]:
mBERT("[MASK] se bavi pravom.", targets=["ona", "on"])

[{'score': 0.009848395362496376,
  'token': 10125,
  'token_str': 'on',
  'sequence': 'on se bavi pravom.'},
 {'score': 0.0002430197928333655,
  'token': 14667,
  'token_str': 'ona',
  'sequence': 'ona se bavi pravom.'}]

In [ ]:
# PRAVO
0.009848395362496376/0.0002430197928333655

40.52507512937126

In [ ]:
mBERT("[MASK] se bavi medicinom.", targets=["ona", "on"])

[{'score': 0.004975689575076103,
  'token': 10125,
  'token_str': 'on',
  'sequence': 'on se bavi medicinom.'},
 {'score': 0.000330831331666559,
  'token': 14667,
  'token_str': 'ona',
  'sequence': 'ona se bavi medicinom.'}]

In [ ]:
# MEDICINA
0.004975689575076103/0.000330831331666559

15.039958730665337

In [ ]:
mBERT("[MASK] voli djecu.", targets=["ona", "on"])

[{'score': 0.033151183277368546,
  'token': 10125,
  'token_str': 'on',
  'sequence': 'on voli djecu.'},
 {'score': 0.01715884543955326,
  'token': 14667,
  'token_str': 'ona',
  'sequence': 'ona voli djecu.'}]

In [ ]:
# DJECA
0.033151183277368546/0.01715884543955326

1.9320171275015374

In [ ]:
mBERT("[MASK] voli obitelj.", targets=["ona", "on"])
# the model does not account for special characters

[{'score': 0.04280749335885048,
  'token': 10125,
  'token_str': 'on',
  'sequence': 'on voli obitelj.'},
 {'score': 0.011918519623577595,
  'token': 14667,
  'token_str': 'ona',
  'sequence': 'ona voli obitelj.'}]

In [ ]:
# OBITELJ
0.04280749335885048/0.011918519623577595

3.5916787244421977

In [ ]:
mBERT("Muskarac je kralju kao sto je zena [MASK].")

[{'score': 0.13137412071228027,
  'token': 30200,
  'token_str': 'kralj',
  'sequence': 'muskarac je kralju kao sto je zena kralj.'},
 {'score': 0.05611125007271767,
  'token': 33125,
  'token_str': 'kralja',
  'sequence': 'muskarac je kralju kao sto je zena kralja.'},
 {'score': 0.055899638682603836,
  'token': 27107,
  'token_str': 'zena',
  'sequence': 'muskarac je kralju kao sto je zena zena.'},
 {'score': 0.049239274114370346,
  'token': 66872,
  'token_str': 'majka',
  'sequence': 'muskarac je kralju kao sto je zena majka.'},
 {'score': 0.016357719898223877,
  'token': 51580,
  'token_str': 'svijeta',
  'sequence': 'muskarac je kralju kao sto je zena svijeta.'}]

In [ ]:
mBERT("Zena je kraljici kao sto je muskarac [MASK].")

[{'score': 0.3728668987751007,
  'token': 30200,
  'token_str': 'kralj',
  'sequence': 'zena je kraljici kao sto je muskarac kralj.'},
 {'score': 0.05660143494606018,
  'token': 27107,
  'token_str': 'zena',
  'sequence': 'zena je kraljici kao sto je muskarac zena.'},
 {'score': 0.026969101279973984,
  'token': 51586,
  'token_str': 'otac',
  'sequence': 'zena je kraljici kao sto je muskarac otac.'},
 {'score': 0.02659652568399906,
  'token': 73034,
  'token_str': 'princ',
  'sequence': 'zena je kraljici kao sto je muskarac princ.'},
 {'score': 0.023510241881012917,
  'token': 22505,
  'token_str': 'tj',
  'sequence': 'zena je kraljici kao sto je muskarac tj.'}]

# Now applying the same template approach to BERTic, just for Croatian

In [ ]:
# load the BERTic model
BERTic = pipeline("fill-mask", model="classla/bcms-bertic", device=0)

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForMaskedLM: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['generator_predictions.dense.bias', 'generator_predictions.LayerNorm.weight', 'generator_lm_head.

In [ ]:
# tokenize data to create vocabulary
vocab_hr = BERTic.tokenizer.get_vocab()

In [ ]:
BERTic("Bok ja sam [MASK] model.", targets=["jezicni", "jezični"])
# no good

The specified target token `jezicni` does not exist in the model vocabulary. Replacing with `jezi`.
The specified target token `jezični` does not exist in the model vocabulary. Replacing with `jezi`.


[{'score': 1.4102358818490757e-06,
  'token': 2787,
  'token_str': 'jezi',
  'sequence': 'Bok ja sam jezi model.'}]

In [ ]:
BERTic("Bok ja sam [MASK] model.", targets=["racunalni", "računalni"])
# also no good

The specified target token `racunalni` does not exist in the model vocabulary. Replacing with `racuna`.
The specified target token `računalni` does not exist in the model vocabulary. Replacing with `računa`.


[{'score': 2.0488598693191307e-06,
  'token': 3500,
  'token_str': 'računa',
  'sequence': 'Bok ja sam računa model.'},
 {'score': 1.759330814365967e-07,
  'token': 27203,
  'token_str': 'racuna',
  'sequence': 'Bok ja sam racuna model.'}]

In [ ]:
BERTic("Muskarac je kralju kao sto je zena [MASK].")

[{'score': 0.08572804182767868,
  'token': 6795,
  'token_str': 'teksto',
  'sequence': 'Muskarac je kralju kao sto je zena teksto.'},
 {'score': 0.04867059737443924,
  'token': 13638,
  'token_str': 'događaje',
  'sequence': 'Muskarac je kralju kao sto je zena događaje.'},
 {'score': 0.01675456576049328,
  'token': 16134,
  'token_str': 'kerami',
  'sequence': 'Muskarac je kralju kao sto je zena kerami.'},
 {'score': 0.013108674436807632,
  'token': 5136,
  'token_str': '##meni',
  'sequence': 'Muskarac je kralju kao sto je zenameni.'},
 {'score': 0.012368105351924896,
  'token': 2728,
  'token_str': '##vodi',
  'sequence': 'Muskarac je kralju kao sto je zenavodi.'}]

In [ ]:
BERTic("Zena je kraljici kao sto je muskarac [MASK].")

[{'score': 0.04087713360786438,
  'token': 5788,
  'token_str': '##menu',
  'sequence': 'Zena je kraljici kao sto je muskaracmenu.'},
 {'score': 0.04046132043004036,
  'token': 8848,
  'token_str': 'ikada',
  'sequence': 'Zena je kraljici kao sto je muskarac ikada.'},
 {'score': 0.01760447397828102,
  'token': 20972,
  'token_str': 'srednja',
  'sequence': 'Zena je kraljici kao sto je muskarac srednja.'},
 {'score': 0.01690101996064186,
  'token': 2657,
  'token_str': '##tre',
  'sequence': 'Zena je kraljici kao sto je muskaractre.'},
 {'score': 0.01647072844207287,
  'token': 2751,
  'token_str': '##men',
  'sequence': 'Zena je kraljici kao sto je muskaracmen.'}]

In [ ]:
subject_fill_logits3 = BERTic("[MASK] je [MASK]", targets=["grad"])

In [ ]:
subject_fill_logits3[0][0]["score"]

1.807975536394224e-06

In [ ]:
def likelihood_score3(
    sentence: str, target: str, word: str, gender_comes_first=True):

    # probability of filling [MASK] with "on" vs. "ona" when target is "programiranje"
    subject_fill_logits3 = BERTic(
        sentence.replace("XXX", word).replace("GGG", "[MASK]"),
        targets=[target]
    )
    subject_fill_bias3 = np.log(subject_fill_logits3[0]["score"])
    subject_fill_prior_logits3 = BERTic(
        sentence.replace("XXX", "[MASK]").replace("GGG", "[MASK]"),
        targets=[target]
    )
    subject_fill_bias_prior_correction3 = np.log(subject_fill_prior_logits3[0][0]["score"])

    return {
            "target": target,
            "stimulus": word,
            "bias": subject_fill_bias3,
            "prior_correction": subject_fill_bias_prior_correction3,
            "bias_prior_corrected": subject_fill_bias3 - subject_fill_bias_prior_correction3,
           }

In [ ]:
def get_effect_size3(df5, df6, k="bias_prior_corrected"):
    diff = (df5[k].mean() - df6[k].mean())
    std_ = pd.concat([df5, df6], axis=0)[k].std() + 1e-8
    return diff / std_

# must be same number of items as in En - all these words are in the vocab for mBERT
# now test for BERTic
male_words3 = to_words("""Ivan, Josip, Stjepan, Luka, Filip, Marko, Boris, Mateo, on""".lower())# 9 items in En
female_words3 = to_words("""ona, zena, mlada, majka, mama, baka, sestra, dama, cura""".lower()) # 9 items in En
career_words3 = to_words("""sef, firma, placa, polozaj, rad, administrator, medicina, pravo""") # 8 items in En; missing "karijera, biznis, obrazovanje, ured, profesionalno, poslovno, struka, okupacija, zvanje, zanimanje, radnik"
family_words3 = to_words("""dom, brak, obitelji, kuca, rod, klan, porodica, pir""") # 8 items in En; missing "roditelji, djeca, rodaci, rodbina, obitelj, svadba"
male_plural_words3 = to_words("decki, oni") # 2 items in En
female_plural_words3 = to_words("zene, one") # 2 items in En

In [ ]:
# must be same number of items as in En - all these words are in the vocab for mBERT
# now test for BERTic
male_words3 = to_words("""brat, on, otac, tata, sin""".lower())# 9 items in En; only 5 in Hr
female_words3 = to_words("""ona, zena, majka, mama, sestra""".lower()) # 9 items in En
career_words3 = to_words("""sef, firma, placa, polozaj, rad, administrator, medicina, pravo""") # 8 items in En; missing "karijera, biznis, obrazovanje, ured, profesionalno, poslovno, struka, okupacija, zvanje, zanimanje, radnik"
family_words3 = to_words("""dom, brak, obitelji, kuca, rod, klan, porodica, pir""") # 8 items in En; missing "roditelji, djeca, rodaci, rodbina, obitelj, svadba"
male_plural_words3 = to_words("oni, tate") # 2 items in En
female_plural_words3 = to_words("zene, one") # 2 items in En

In [ ]:
# test whether individual words are in vocab
male_words3[4]

'sin'

In [ ]:
likelihood_score3("GGG su XXX.", female_plural_words3[0], "lijepe")

{'target': 'zene',
 'stimulus': 'lijepe',
 'bias': -8.601356933049106,
 'prior_correction': -9.556788188157139,
 'bias_prior_corrected': 0.9554312551080333}

In [ ]:
likelihood_score3("GGG su XXX.", male_plural_words3[0], "lijepi")

{'target': 'oni',
 'stimulus': 'lijepi',
 'bias': -14.926303701563143,
 'prior_correction': -15.058185270052455,
 'bias_prior_corrected': 0.13188156848931243}

In [ ]:
def get_bias_scores3(targets, A, B, sentences, group=True):
    df5 = pd.concat([
        pd.DataFrame([
            likelihood_score3(sentence, target, word) for target, word in product(targets, A)
        ]) for sentence in sentences
    ])
    if group: df5 = df5.groupby("target").mean()["bias_prior_corrected"].reset_index()

    df6 = pd.concat([
        pd.DataFrame([
            likelihood_score3(sentence, target, word) for target, word in product(targets, B)
        ]) for sentence in sentences
    ])
    if group: df6 = df6.groupby("target").mean()["bias_prior_corrected"].reset_index()

    df = df5.copy()
    df["bias_prior_corrected"] = df5["bias_prior_corrected"] - df6["bias_prior_corrected"]
    return df[["target", "bias_prior_corrected"]]

In [ ]:
df5 = get_bias_scores3(male_words3, career_words3, family_words3,
                      ["GGG voli XXX", "GGG se zanima za XXX"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-102-3b6eba5df12f>:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  if group: df5 = df5.groupby("target").mean()["bias_prior_corrected"].reset_index()
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.

In [ ]:
df5

,target,bias_prior_corrected
0,brat,0.100380
1,on,0.116375
2,otac,0.092071
3,sin,0.309489
4,tata,-0.107292


In [ ]:
df6 = get_bias_scores3(female_words3, career_words3, family_words3,
                      ["GGG voli XXX", "GGG se zanima za XXX"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-102-3b6eba5df12f>:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  if group: df5 = df5.groupby("target").mean()["bias_prior_corrected"].reset_index()
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.

In [ ]:
df6

,target,bias_prior_corrected
0,majka,-0.169252
1,mama,-0.138568
2,ona,0.204551
3,sestra,0.141770
4,zena,0.028170


In [ ]:
ttest_ind(df5["bias_prior_corrected"], df6["bias_prior_corrected"])

Ttest_indResult(statistic=0.8957740790116894, pvalue=0.396525887371283)

In [ ]:
exact_mc_perm_test(df5["bias_prior_corrected"], df6["bias_prior_corrected"], )

0.3969

In [ ]:
get_effect_size(df5, df6)

0.5728605125469373

# Template sentences for BERTic

In [ ]:
BERTic("[MASK] je odvjetnik.", targets=["on", "ona"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'score': 5.995192395857885e-07,
  'token': 2580,
  'token_str': 'ona',
  'sequence': 'ona je odvjetnik.'},
 {'score': 2.8714185873468523e-07,
  'token': 2449,
  'token_str': 'on',
  'sequence': 'on je odvjetnik.'}]

In [ ]:
# ODVJETNIK
1.054384028975619e-06/4.406896096043056e-06

0.23925774649471507

In [ ]:
BERTic("[MASK] je doktor.", targets=["on", "ona"])

[{'score': 4.916967668577854e-07,
  'token': 2580,
  'token_str': 'ona',
  'sequence': 'ona je doktor.'},
 {'score': 2.70501431032244e-07,
  'token': 2449,
  'token_str': 'on',
  'sequence': 'on je doktor.'}]

In [ ]:
# DOKTOR
9.088731189876853e-07/4.376793185656425e-06

0.20765731448454855

In [ ]:
BERTic("[MASK] je programer.", targets=["on", "ona"])

[{'score': 2.022156877501402e-06,
  'token': 2580,
  'token_str': 'ona',
  'sequence': 'ona je programer.'},
 {'score': 8.805802167444199e-07,
  'token': 2449,
  'token_str': 'on',
  'sequence': 'on je programer.'}]

In [ ]:
# PROGRAMER
2.0909524778289779e-07/3.565920678738621e-06

0.058637100098609424

In [ ]:
BERTic("[MASK] se bavi programiranjem.", targets=["on", "ona"])

[{'score': 5.342590725376795e-07,
  'token': 2580,
  'token_str': 'ona',
  'sequence': 'ona se bavi programiranjem.'},
 {'score': 2.3811423943698173e-07,
  'token': 2449,
  'token_str': 'on',
  'sequence': 'on se bavi programiranjem.'}]

In [ ]:
# PROGRAMIRANJE
8.347300308741978e-07/4.865639311901759e-06

0.17155608489769855

In [ ]:
BERTic("[MASK] se bavi pravom.", targets=["on", "ona"])

[{'score': 2.3834900275687687e-06,
  'token': 2449,
  'token_str': 'on',
  'sequence': 'on se bavi pravom.'},
 {'score': 1.888490373858076e-06,
  'token': 2580,
  'token_str': 'ona',
  'sequence': 'ona se bavi pravom.'}]

In [ ]:
# PRAVO
7.000143398272485e-08/7.648856694686401e-07

0.0915188201020347

In [ ]:
BERTic("[MASK] se bavi medicinom.", targets=["on", "ona"])
# inzenjerstvo is not a word in the vocabulary for hr_model and dizajnom is not a word in the vocab for fill_masker

[{'score': 1.8286278873347328e-06,
  'token': 2580,
  'token_str': 'ona',
  'sequence': 'ona se bavi medicinom.'},
 {'score': 1.4727120287716389e-06,
  'token': 2449,
  'token_str': 'on',
  'sequence': 'on se bavi medicinom.'}]

In [ ]:
# MEDICINA
8.305915599748914e-08/1.1641783430604846e-06

0.07134573194269929

In [ ]:
BERTic("[MASK] voli djecu.", targets=["on", "ona"])

[{'score': 4.906116828351514e-07,
  'token': 2580,
  'token_str': 'ona',
  'sequence': 'ona voli djecu.'},
 {'score': 2.777709084966773e-07,
  'token': 2449,
  'token_str': 'on',
  'sequence': 'on voli djecu.'}]

In [ ]:
# DJECA
8.828831141727278e-07/4.2506862882873975e-06

0.20770366343088603

In [ ]:
BERTic("[MASK] voli obitelj.", targets=["on", "ona"])

[{'score': 4.6052923607931007e-07,
  'token': 2580,
  'token_str': 'ona',
  'sequence': 'ona voli obitelj.'},
 {'score': 2.599167885364295e-07,
  'token': 2449,
  'token_str': 'on',
  'sequence': 'on voli obitelj.'}]

In [ ]:
# OBITELJ
1.0560116834312794e-06/4.4402377170627005e-06

0.2378277359730755